<a href="https://colab.research.google.com/github/Varun9213/CE-903-DEC/blob/main/DEC_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model


from sklearn.datasets import load_iris, load_digits
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
def autoencoder(DIM, act='relu', initializer='glorot_uniform'):

  """
  DIM : Number of units in each layer where DIM[0] corresponds to input layer and DIM[-1] corresponds to encoder output and decoder input
  act : Activation Function

  """
  inp = L.Input(shape=(DIM[0],), name='input_')
  e = inp

  #Encoding Layers
  for i in range(len(DIM)-2):
    e = L.Dense(DIM[i+1], act, kernel_initializer=initializer, name="encoding_layer_%i"%(i))(e)

  e = L.Dense(DIM[-1], kernel_initializer=initializer, name='encoder_output')(e)
  encoder_out = e 

  d = e
  #Decoding Layers 
  for i in range(len(DIM)-2, 0, -1):
    d = L.Dense(DIM[i], act, kernel_initializer=initializer, name="decoding_layer_%i"%(i))(d)

  decoder_out = L.Dense(DIM[0], kernel_initializer=initializer, name='decoder_ouput')(d)

  encoder = Model(inp,encoder_out,name="Encoder")
  autoencoder = Model(inp,decoder_out,name="Autoencoder")

  return autoencoder, encoder

In [ ]:
class ClusteringLayer(tf.keras.layers.Layer):

  def __init__(self,nclusters,weights=None,alpha=1.0,**kwargs):
    if 'input_shape' not in kwargs and 'input_dim' in kwargs:
      kwargs['input_shape'] = (kwargs.pop('input_dim'),)
    super().__init__(**kwargs)
    self.nclusters = nclusters
    self.init_weights = weights
    self.alpha = alpha

  def build(self, input_shape):
    self.inp_dim = input_shape[1]
    self.clusters = self.add_weight(shape=(self.nclusters,self.inp_dim),initializer='glorot_uniform', name='clusters')
    if self.init_weights is not None:
      self.set_weights(self.init_weights)
      self.built = True

  def call(self, inputs, **kwargs):
    q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
    q = K.pow(q,(self.alpha + 1.0) / 2.0)
    q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
    return q


In [ ]:
class DEC(object):

  def __init__(self,DIM,nclusters=10,alpha=1.0,initializer='glorot_uniform'):
    self.DIM = DIM
    self.input_dim = DIM[0]
    self.nclusters = nclusters
    self.alpha = alpha
    self.autoencoder, self.encoder = autoencoder(DIM, initializer=initializer)

    clustering_layer = ClusteringLayer(self.nclusters, name='cluster_layer')(self.encoder.output)
    self.model = Model(inputs=self.encoder.input, outputs=clustering_layer)

  def train_autoencoder(self, x, optimizer='adam', epochs=100, batch_size=256, dir = None):

    self.autoencoder.compile(optimizer=optimizer, loss='mse')
    hist = self.autoencoder.fit(x,x,epochs=epochs,batch_size=batch_size,verbose=0)
    print("loss : %.5f"%hist.history["loss"][-1])
    self.autoencoder.save_weights(dir + "/autoencoder_weights.h5")
    print("Weights saved to %s/autoencoder_weights.h5" %(dir))


  def target_distribution(self,q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

  def compile(self, optimizer='sgd', loss='kld'):
    self.model.compile(optimizer=optimizer, loss=loss)

  def predict(self, x):
    q = self.model.predict(x)
    return np.argmax(q,axis=1)

  def fit(self, x, y=None, epochs=200, batch_size=256, dir = None):

    print('Initializing cluster centers with k-means.........')
    km = KMeans(n_clusters=self.nclusters, n_init=5)
    y_pred = km.fit_predict(self.encoder.predict(x))
    self.model.summary()
    self.model.get_layer('cluster_layer').set_weights([km.cluster_centers_])
    loss = 0
    indx = 0
    idx_arr = np.arange(x.shape[0])

    for i in range(epochs):

      q = self.model.predict(x)
      p = self.target_distribution(q)
      idx = idx_arr[indx*batch_size:min((indx+1) * batch_size, x.shape[0])]
      loss = self.model.train_on_batch(x[idx],p[idx])
      indx = indx + 1 if (indx + 1) * batch_size <= x.shape[0] else 0
    print('Loss : %.5f'%loss)
    print('saving model to:', dir + "/DEC_model.h5")
    self.model.save_weights(dir + "/DEC_model.h5")


In [ ]:
fea, tar = load_digits(return_X_y=True)
tar

array([0, 1, 2, ..., 8, 9, 8])

In [ ]:
fea

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [ ]:
fea = np.array(fea)
tar = np.array(tar)

In [ ]:
%mkdir iris_DEC

In [ ]:
dec = DEC([fea.shape[1],128,64,64,32,32],nclusters=10)
dec.autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
decoding_layer_4 (Dense)     (None, 32)                

In [ ]:
dec.train_autoencoder(fea,epochs=5000,dir="/content/iris_DEC",optimizer=Adam(learning_rate=0.001))

loss : 0.64358
Weights saved to /content/iris_DEC/autoencoder_weights.h5


In [ ]:
dec.compile(optimizer=Adam(learning_rate=0.001), loss='kld')
dec.fit(fea, tar, epochs=1000, dir="/content/iris_DEC")

Initializing cluster centers with k-means.........
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
cluster_

In [ ]:
y_pred = dec.predict(fea)

In [ ]:
cm = metrics.cluster.contingency_matrix(tar,y_pred)
cm

array([[  0,   0,   0,   1,   0,   0,   0, 177,   0,   0],
       [  0,   0,   0,   0,   0,  98,  27,   0,  57,   0],
       [  0,   0,   0,   0,   0,   9, 168,   0,   0,   0],
       [  1, 168,  11,   0,   3,   0,   0,   0,   0,   0],
       [  0,   0,   0, 175,   3,   3,   0,   0,   0,   0],
       [176,   3,   1,   0,   1,   0,   0,   0,   0,   1],
       [  0,   0,   2,   0,   0,   0,   0,   2,   0, 177],
       [  0,   0,   1,   0, 178,   0,   0,   0,   0,   0],
       [  0,   1, 156,   0,   1,  12,   2,   0,   2,   0],
       [  0, 145,   3,   0,  11,   0,   0,   0,  21,   0]])

In [ ]:
def acc_clustering(cm,y):
  df = pd.DataFrame(cm)
  true = 0
  max_val = []
  for col in df:
    sorted = df[col].sort_values(ascending = False)
    i = 0
    while sorted.index[i] in max_val:
      i += 1
    true += sorted.values[i]
    max_val.append(sorted.index[i])

  return (true/len(y))*100


In [ ]:
acc_clustering(cm,tar)

83.13856427378965

In [ ]:
y_pred

array([7, 5, 5, ..., 2, 1, 2])

In [ ]:
tar

array([0, 1, 2, ..., 8, 9, 8])